# ANEXO 02

## Preparación de la base de datos

En este documento se procede a crear la variable Cartera_Acu, la cual vino dada de forma que el primer valor de cada escenario es correcto, y a partir de ahí, el valor presentado en la columna es el valor neto de mes.

Cargamos las librerias

In [1]:
#General
import pandas as pd
import numpy as np
from datetime import datetime

Y fijamos el directorio de trabajo

In [2]:
data = pd.read_csv('../dataframes/Actividad_comercial_v01.csv')

## Recálculo columna Carteras

Antes de realizar el análisis de la base de datos, debemos preparar la misma.
El cliente nos indicó que la columna de carteras final no estaba correctamente calculada, por lo que se recalcula como:
    
$$ Cartera_{n} = Cartera_{n-1} +  Altas_{n} - Bajas_{n} $$

Siendo n el mes tratado
    
Hay que hace distinción entre cada combinación de variables. Para solventar este problema generamos una base con los campos únicos y rellenaremos con la info que tengamos de cada mes. En caso de no haberla, rellenamos con 0s

Modificamos las variables al tipo correspondiente:    

In [3]:
for i in data.columns[0:10]:
    data[i] = data[i].astype('object')
    
data['Fecha_Mes'] = pd.to_datetime(data.Fecha_Mes)

In [4]:
data.rename(columns = {'Neto M':'Neto_M', 'Bajas M' : 'Bajas_M', 'Altas M': 'Altas_M'}, inplace = True)

In [5]:
#Verificamos que todos tenemos los mismos tipos de datos
if (all(data.select_dtypes(include='object').columns == pd.Series(['ID_SOCIEDAD', 'ID_SEGMENTO', 'SEGMENTO2', 'ID_LINEA_PRODUCTO_AGREG',
       'ID_AGRUP_PRODUCTO', 'ID_AGRUP_PROD', 'ID_AGRUP_CANAL', 'ID_ZONA',
       'Año', 'ID_MES'])) == True and 
    all(data.select_dtypes(include='datetime64').columns == pd.Series(['Fecha_Mes'])) == True and
    all(data.select_dtypes(include='int64').columns == pd.Series(['Altas_M', 'Altas_Mes_C+T', 'Altas_Mes_Tra', 'Bajas_M', 'Bajas_Mes_C+T',
       'Bajas_Mes_Tra', 'Neto_M', 'Cartera_Acu'])) == True):
   print('OK, las variables están en el estado correcto')
else:
    print('Hay que asignar las variables al estado correcto para continuar con el EDA.')
    # para verificar el estado utilizar dtypes: data.dtypes
    # para cambiar el estado utilizar astype: df['col'].astype(dtype)

OK, las variables están en el estado correcto


Separamos nuestras variables

In [6]:
# Variables numéricas
data_num = data.select_dtypes(np.number)
# Variables cualitativas
data_cual = data.select_dtypes(np.object_)
# Variable tiempo
data_tmp = data.select_dtypes(np.datetime64)

### Generación base

Cogemos nuestras variables cualitativas y nos quedamos con la combinación de estas única sin tener en cuenta las fechas. 

In [7]:
# Generamos la base de cualitativas
base_data = data_cual[data_cual.columns[0:8]].drop_duplicates()
# Nos cargamos el índice
base_data.reset_index(inplace = True, drop = True)
# Valores bucle
index_mes = data_cual['ID_MES'].unique()

Generación del data frame vacío

In [8]:
union = pd.DataFrame()
for i in index_mes:
    base_temp = base_data
    base_temp['ID_MES'] = i
    # Pegamos los valores cuantitativos y fechas que faltan
    df_temp = pd.merge(base_temp, 
                       data,
                       how ='left',
                       on = ['ID_SOCIEDAD', 
                             'ID_SEGMENTO', 
                             'SEGMENTO2', 
                             'ID_LINEA_PRODUCTO_AGREG',
                             'ID_AGRUP_PRODUCTO', 
                             'ID_AGRUP_PROD', 
                             'ID_AGRUP_CANAL', 
                             'ID_ZONA',
                             'ID_MES']
                      )
    # Unimos la base con las anteriores
    union = pd.concat([union, df_temp])
    
union = union.fillna('0.0')

In [9]:
# Recalificamos las variables cualitativas y numéricas
for i in data_cual.columns:
    union[i] = union[i].astype('object')

for i in data_num.columns:
    union[i] = union[i].astype('float')

   ¿Seguimos teniendo las mismas altas y bajas?

In [10]:
data['Neto_M'].agg(['sum']) - union['Neto_M'].agg(['sum'])

sum    0.0
Name: Neto_M, dtype: float64

### Cálculo

Vamos a provechar los índices repetidos que tenemos para el cálculo del total. 
    
Como la estructura está repetida es fácil relacionar cada uno de los totales

In [11]:
# Cargamos los datos
data_prov = union
# Reseteamos el índice
data_prov.reset_index(inplace = True, drop = True)
# Data frame vacío para ir pegando el correcto
Cartera_Acu_fin = pd.DataFrame()

In [12]:
# Columnas merge
col_merge = list(data_cual.columns.drop('Año').drop('ID_MES')) + ['Cartera_Acu']
# Columnas final
col_fin = list(data_prov.columns)
for i, j in enumerate(index_mes):

    # Cogemos los datos de cada mes
    df_tmp = data_prov[data_prov['ID_MES'] == j]
    # Columnas D
    
    
    if i == 0:
        # El primer més lo consideramos como el correcto
        Cartera_Acu_fin = pd.concat([Cartera_Acu_fin, df_tmp])
    else:
        # Cargamos los valores de cartera del mes anterior
        df_tmp_ant = Cartera_Acu_fin[Cartera_Acu_fin['ID_MES'] == mes_ant][col_merge]
        # Renombramos la columna nueva
        df_tmp_ant = df_tmp_ant.rename(columns = {'Cartera_Acu': 'Cartera_Acu_mes_ant'})
        # Unimos los Data frames para pegar la cartera antigua
        df_tmp = pd.merge(df_tmp, df_tmp_ant, 
                          how = 'left',
                          on = ['ID_SOCIEDAD', 
                                'ID_SEGMENTO', 
                                'SEGMENTO2', 
                                'ID_LINEA_PRODUCTO_AGREG',
                                'ID_AGRUP_PRODUCTO', 
                                'ID_AGRUP_PROD', 
                                'ID_AGRUP_CANAL', 
                                'ID_ZONA']
                         )
        # Calculamos la cartera nueva
        df_tmp['Cartera_Acu'] = df_tmp['Cartera_Acu_mes_ant']  + df_tmp['Altas_M'] + df_tmp['Bajas_M']
        # Limpiamos las columnas que sobran
        df_tmp = df_tmp[col_fin]
        # Unimos los data frames
        Cartera_Acu_fin = pd.concat([Cartera_Acu_fin, df_tmp])
    
    # Guardamos el mes anterior
    mes_ant = j
    
Cartera_Acu_fin.reset_index(inplace = True, drop = True)

data_cartera = Cartera_Acu_fin

data_cartera.head(4)

,ID_SOCIEDAD,ID_SEGMENTO,SEGMENTO2,ID_LINEA_PRODUCTO_AGREG,ID_AGRUP_PRODUCTO,ID_AGRUP_PROD,ID_AGRUP_CANAL,ID_ZONA,ID_MES,Año,Fecha_Mes,Altas_M,Altas_Mes_C+T,Altas_Mes_Tra,Bajas_M,Bajas_Mes_C+T,Bajas_Mes_Tra,Neto_M,Cartera_Acu
0,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,201912,2019,2019-12-31 00:00:00,4.0,4.0,0.0,-1.0,-1.0,0.0,3.0,131.0
1,SOC02,SEG01,LP01,1,1,PROD02,1,ZONA01,201912,2019,2019-12-31 00:00:00,4.0,4.0,0.0,-3.0,-3.0,0.0,1.0,630.0
2,SOC02,SEG01,LP01,1,1,PROD09,1,ZONA01,201912,2019,2019-12-31 00:00:00,3.0,3.0,0.0,-163.0,-163.0,0.0,-160.0,20503.0
3,SOC02,SEG02,LP01,1,1,PROD02,1,ZONA01,201912,2019,2019-12-31 00:00:00,18.0,18.0,0.0,-73.0,-73.0,0.0,-55.0,4167.0


### Corección Columna Fecha Mes

La columna "Fecha Mes" se nos ha corrompido un poco al generar la base
    
Generamos un diccionario con las relaciones entre los ID_MES que si está correcto y la fecha mes

In [13]:
# Generamos la base de la relación de fechas
base_fecha = data_cartera[data_cartera['Fecha_Mes']!='0.0'][['ID_MES', 'Fecha_Mes', 'Año']].drop_duplicates()
# Cambiamos el nombre de la columna
base_fecha = base_fecha.rename(columns = {'Fecha_Mes': 'Fecha_Mes_new',
                                          'Año': 'Año_new'})
# Pegamos la fecha corregida por la derecha
data_cartera = pd.merge(data_cartera, 
                     base_fecha, 
                     how = 'left',
                     on = ['ID_MES']
                 )
# Sustituimos la fecha antigua por la nueva
data_cartera['Fecha_Mes'] = data_cartera['Fecha_Mes_new']
# Sustituimos el año antiguo por el nuevo
data_cartera['Año'] = data_cartera['Año_new']
# Eliminamos la columna introducida
data_cartera = data_cartera[col_fin]
# Corregimos el tipo de dato a datetime
data_cartera['Fecha_Mes'] = pd.to_datetime(data_cartera.Fecha_Mes)

# Reseteamos el índice
data_cartera.reset_index(inplace = True, drop = True)

data_cartera.head(4)

,ID_SOCIEDAD,ID_SEGMENTO,SEGMENTO2,ID_LINEA_PRODUCTO_AGREG,ID_AGRUP_PRODUCTO,ID_AGRUP_PROD,ID_AGRUP_CANAL,ID_ZONA,ID_MES,Año,Fecha_Mes,Altas_M,Altas_Mes_C+T,Altas_Mes_Tra,Bajas_M,Bajas_Mes_C+T,Bajas_Mes_Tra,Neto_M,Cartera_Acu
0,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,201912,2019,2019-12-31,4.0,4.0,0.0,-1.0,-1.0,0.0,3.0,131.0
1,SOC02,SEG01,LP01,1,1,PROD02,1,ZONA01,201912,2019,2019-12-31,4.0,4.0,0.0,-3.0,-3.0,0.0,1.0,630.0
2,SOC02,SEG01,LP01,1,1,PROD09,1,ZONA01,201912,2019,2019-12-31,3.0,3.0,0.0,-163.0,-163.0,0.0,-160.0,20503.0
3,SOC02,SEG02,LP01,1,1,PROD02,1,ZONA01,201912,2019,2019-12-31,18.0,18.0,0.0,-73.0,-73.0,0.0,-55.0,4167.0


### Guardamos el dataset en el entorno virtual

In [14]:
%store data_cartera

Stored 'data_cartera' (DataFrame)


En este punto tenemos la variable "Cartera_Acu" se encuentra con los valores corregidos
    
Las variables están en el formato que les corresponde